In [2]:
!pip install groq python-dotenv --quiet

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from groq import Groq
import os

client = Groq(api_key=os.environ["GROQ_API_KEY"])


In [13]:
MODEL_NAME = "llama-3.3-70b-versatile"

MODEL_CONFIG = {
    "technical": {
        "system_prompt": """
You are a Technical Support Expert.
Be precise, logical, and code-focused.
Explain bugs clearly and provide fixes with examples.
Avoid casual conversation.
"""
    },

    "billing": {
        "system_prompt": """
You are a Billing Support Specialist.
Be empathetic and professional.
Focus on payments, refunds, subscriptions, and policies.
Give clear next steps to resolve billing issues.
"""
    },

    "general": {
        "system_prompt": """
You are a friendly customer support assistant.
Handle general questions and casual conversation politely.
"""
    }
}


In [14]:
def route_prompt(user_input):

    routing_prompt = f"""
Classify the following customer query into ONE category:

Categories:
- technical
- billing
- general
- tool

Return ONLY the category word.

Query:
{user_input}
"""

    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0,  # deterministic routing
        messages=[
            {"role": "system", "content": "You are an intent classifier."},
            {"role": "user", "content": routing_prompt}
        ]
    )

    category = response.choices[0].message.content.strip().lower()
    return category


In [15]:
def get_bitcoin_price():
    # Mock API response (assignment requirement)
    return "The current Bitcoin price is approximately $65,000 USD (mock data)."


In [16]:
def call_expert(system_prompt, user_input):

    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0.7,  # creativity for experts
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]
    )

    return response.choices[0].message.content


In [17]:
def process_request(user_input):

    # Step 1: Route request
    category = route_prompt(user_input)
    print(f"🔀 Routed to: {category}")

    # Step 2: Tool handling (BONUS)
    if category == "tool":
        return get_bitcoin_price()

    # Step 3: Fallback safety
    if category not in MODEL_CONFIG:
        category = "general"

    # Step 4: Load expert prompt
    system_prompt = MODEL_CONFIG[category]["system_prompt"]

    # Step 5: Call expert
    answer = call_expert(system_prompt, user_input)

    return answer


In [18]:
print(process_request(
    "My python script throws IndexError on line 5"
))


🔀 Routed to: technical
To assist you with the `IndexError` on line 5 of your Python script, I need more information about the code. 

**Please provide the following:**
1. The code snippet that includes line 5.
2. The full error message you're receiving.

Without this information, it's challenging to identify the root cause of the issue. However, I can provide a general explanation of `IndexError` and a common fix.

**IndexError Explanation:**
An `IndexError` occurs when you attempt to access an element in a sequence (like a list, tuple, or string) using an index that is out of range.

**Common Fix:**
To fix an `IndexError`, ensure that the index you're using is within the bounds of the sequence. You can do this by checking the length of the sequence before accessing an element.

**Example:**
```python
# Incorrect code that may throw IndexError
my_list = [1, 2, 3]
print(my_list[3])  # This will throw IndexError because my_list only has 3 elements (index 0 to 2)

# Corrected code
my_list

In [19]:
print(process_request(
    "I was charged twice for my subscription"
))


🔀 Routed to: billing
I'm so sorry to hear that you were charged twice for your subscription. I can imagine how frustrating that must be for you.

To resolve this issue, I'd like to assist you in getting a refund for the duplicate charge. Can you please provide me with some more information about the issue? Specifically, I'll need to know:

1. Your subscription ID or account name
2. The date of the duplicate charge
3. The amount of the duplicate charge

Once I have this information, I'll be happy to investigate further and process a refund for the duplicate charge.

Additionally, I'll also review your account to ensure that the duplicate charge was an error and that your subscription is set up correctly to prevent any future duplicate charges.

If you have any receipts or transaction records related to the duplicate charge, please feel free to share them with me as well.

Let's work together to resolve this issue as quickly as possible. What's the best way for me to contact you if I nee

In [20]:
print(process_request(
    "Hi, what services do you offer?"
))


🔀 Routed to: general
Hello. It's nice to meet you. We offer a wide range of services to help with your needs. Our main areas of support include answering general questions, providing information on various topics, and assisting with basic troubleshooting for common issues.

If you have a specific question or problem in mind, feel free to let me know and I'll do my best to help. We're here to provide helpful and friendly support, so please don't hesitate to ask anything.

What brings you here today? Is there something in particular you'd like to know or discuss?


In [21]:
print(process_request(
    "What is the current price of Bitcoin?"
))


🔀 Routed to: general
I'd be happy to help you with that. However, please note that the price of Bitcoin can fluctuate rapidly and may have changed since my knowledge was last updated.

As of my knowledge cutoff, I don't have real-time access to current market prices. But I can suggest some ways for you to find the current price of Bitcoin. You can check a reliable cryptocurrency website, such as CoinMarketCap or Coinbase, for the most up-to-date pricing information. Alternatively, you can also check financial news websites or mobile apps that provide real-time market data.

If you're looking for general information about Bitcoin or have any other questions, I'd be happy to help!
